In [5]:
import xml.etree.ElementTree as ET
import re

def to_tei(xml_entree, chemin_sortie):
    
    transcriptions = {}
    types_a_enlever = ['signature', 'page', 'marginalia', 'header']
    
    entree_tree = ET.parse(xml_entree)
    entree_root = entree_tree.getroot()
    
    # DEBUT DE L'ECRITURE DU XML DE SORTIE
    
    # Création de la racine.
    sortie_root = ET.Element("TEI")
    
    # Récupération du modèle de teiHeader depuis un fichier externe.
    fichier_teiHeader = ET.parse('data/teiHeader.xml')
    teiHeader = fichier_teiHeader.getroot()
    sortie_root.append(teiHeader)
    
    # Faire un dictionnaire avec les identifiants.
    for ligne in entree_root.findall('.//l'):
        cle = ligne.get('facs')
        transcriptions[cle] = str(ligne.text)
    
    
    # Création des éléments facsimile, un par un.
    for facsimile in entree_root.findall('facsimile'):
        
        for zone in facsimile.findall('surface/zone[@rendition="TextRegion"]'):
            if zone.findall('zone[@rendition="Line"]'):
                
                for ligne in zone.findall('zone[@rendition="Line"]'):
                
                    if zone.get('subtype') == "page":
                        a_trouver = "#" + str(ligne.get('identifiant'))
                        fw = ET.Element('fw', subtype='pageNum', place='top')
                        fw.text = transcriptions[a_trouver]
                        ligne.append(fw)
                    
                    elif zone.get('subtype') == "header":
                        a_trouver = "#" + str(ligne.get('identifiant'))
                        fw = ET.Element('fw', subtype='head', place='top-centre')
                        fw.text = transcriptions[a_trouver]
                        ligne.append(fw)
                    
                    elif zone.get('subtype') == "marginalia":
                        a_trouver = "#" + str(ligne.get('identifiant'))
                        fw = ET.Element('fw', subtype='marginalia', place='side')
                        fw.text = transcriptions[a_trouver]
                        ligne.append(fw)
                    
            
                    elif zone.get('subtype') == "signature":
                        a_trouver = "#" + str(ligne.get('identifiant'))
                        fw = ET.Element('fw', subtype='sig', place='bot-right')
                        fw.text = transcriptions[a_trouver]
                        ligne.append(fw)
                        
                   
        sortie_root.append(facsimile)
    
    text = ET.Element('text')
    front = ET.Element('front')
    body = ET.Element('body')
    back = ET.Element('back')
    
    ancien_front_repris = entree_root.find('text/front')
    ancien_body_repris = entree_root.find('text/body')
    
    chapitre = ET.Element('div', subtype='chapitre')
    section = ET.Element('div', subtype='section')
    
    for para in ancien_front_repris.findall('.//p'):
        
        if para.get('subtype') == "titre":
            
            if chapitre.findall('.//p'):
                front.append(chapitre)
                del chapitre
                chapitre = ET.Element('div', subtype='chapitre')

            head = ET.Element('head')
            for ligne in para.findall('.//l'):
                head.append(ligne)
            chapitre.append(head)
            
        elif para.get('subtype') == "coutume" or para.get('subtype') == "note_interne":
            paragraphe = ET.Element('p')
            paragraphe.set('subtype', para.get('subtype'))
            for ligne in para.findall('.//l'):
                paragraphe.append(ligne)
            chapitre.append(paragraphe)
            
        ancien_front_repris.remove(para)
    
    
    front.append(chapitre)
    del chapitre
    chapitre = ET.Element('div', subtype='chapitre')
    
    
    for parag in ancien_body_repris.findall('.//p'):
        
        # Si c'est un titre, on ferme le chapitre et on l'ajoute au FRONT.
        if parag.get('subtype') == "titre":
            
            if section.findall('.//p'):
                chapitre.append(section)
                del section
                section = ET.Element('div', subtype='section')
                
            if chapitre.findall('.//p'):
                body.append(chapitre)
                del chapitre
                chapitre = ET.Element('div', subtype='chapitre')
            
            head = ET.Element('head')
            for ligne in parag.findall('.//l'):
                head.append(ligne)
            chapitre.append(head)
            
            
        elif parag.findall('.//l[@subtype="sectionhead"]'):
            if section.findall('.//p'):
                chapitre.append(section)
                del section
                section = ET.Element('div', subtype='section')
            
            # Formation et ajout du titre de section.
            facs = parag.find('.//l[@subtype="sectionhead"]').get('facs')
            head = ET.Element('head')
            head.set('facs', facs)
            head.text = parag.find('.//l[@subtype="sectionhead"]').text
            section.append(head)
            
            # Formation et ajout du paragraphe suivant le titre.
            quote = ET.Element('quote', subtype='coutume')
            paragraphe = ET.SubElement(quote, 'p')
            for ligne in parag.findall('.//l'):
                paragraphe.append(ligne)
            section.append(quote)
           
        else:
            if parag.get('subtype') == "coutume":
                quote = ET.Element('quote', subtype='coutume')
                paragraphe = ET.SubElement(quote, 'p')
                for ligne in parag.findall('.//l'):
                    paragraphe.append(ligne)
                section.append(quote)
           
            elif parag.get('subtype') == "note_interne":
                paragraphe = ET.Element('p', subtype='note_interne')
                for ligne in parag.findall('.//l'):
                    paragraphe.append(ligne)
                section.append(paragraphe)
                
            elif parag.get('subtype') == "note_de_note":
                paragraphe = ET.Element('p', subtype='note_de_note')
                for ligne in parag.findall('.//l'):
                    paragraphe.append(ligne)
                section.append(paragraphe)
                
            elif parag.get('subtype') == "footnote-continued":
                paragraphe = ET.Element('p', subtype='note_de_note_continuee')
                for ligne in parag.findall('.//l'):
                    paragraphe.append(ligne)
                section.append(paragraphe)
                
            elif parag.get('subtype') == None:
                paragraphe = ET.Element('p')
                for ligne in parag.findall('.//l'):
                    paragraphe.append(ligne)
                back.append(paragraphe)
            
        ancien_body_repris.remove(parag)
    
    # Ne pas oublier d'ajouter la dernière section au chapitre,
    # et le dernier chapitre au body.
    chapitre.append(section)
    body.append(chapitre)
    
    
    text.append(front)
    text.append(body)
    text.append(back)
    
    sortie_root.append(text)
    
    a_ecrire = ET.tostring(sortie_root, encoding="unicode", method="xml")
    
    ecriture = open(chemin_sortie, "w")
    ecriture.write(a_ecrire)

In [9]:
to_tei('/home/erminea/Documents/CONDE/Ruines_export_TS/TAC_TEI_TS/TAC_TEI_TS_tei.xml',
      '/home/erminea/Documents/CONDE/Ruines_export_TS/TAC_TEI_TS/TAC_transf.xml')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

